In [4]:
import pandas as pd
from pandas.io import gbq
import datetime
from datetime import date

filen = input('please provide the file name you want to import')

# import the txt file downloaded from SC 

df = pd.read_csv(f'/home/markaw/heroes/netsuite_cashsales/sc_flatfiles/{filen}.txt', delimiter = "\t",decimal=",",
                 dtype={"settlement-id": str, 
                        "settlement-start-date": str, 
                        "settlement-end-date": str, 
                        "deposit-date": str,
                        "total-amount": str, 
                        "adjustment-id": str, 
                        "shipment-id": str, 
                        "merchant-order-item-id": str})

# Filter for the elements we need for this particular Netsuite import (cash sales i.e order/principal)

cash_sale = df[(df['transaction-type'] == 'Order')&(df['amount-description'] == 'Principal')|(df['amount-description'] == 'Tax')].copy()

sql_query = f"""
SELECT distinct sku, heroes_product_id FROM analytics_viz.f_amz_order_items
"""

# Map heroes ID, customers and locations, and transsform date information

heroesid = gbq.read_gbq(sql_query, project_id = "analytics-298917")

cs_heroes = cash_sale.merge(heroesid[['heroes_product_id','sku']], left_on='sku', right_on='sku')

cs_heroes['amount'] = pd.to_numeric(cs['amount'])

try: 
    cs_heroes['datetime'] = pd.to_datetime(cs_heroes['posted-date-time'], format='%d.%m.%Y %H:%M:%S %Z')
except:
    cs_heroes['datetime'] = pd.to_datetime(cs_heroes['posted-date-time'], format='%Y-%m-%d %H:%M:%S %Z')
    
cs_heroes['tranDate'] = cs_heroes['datetime'].apply(lambda x: datetime.datetime.strftime(x, "%d/%m/%Y"))
cs_heroes['postingPeriodRef'] =cs_heroes['datetime'].apply(lambda x: datetime.datetime.strftime(x, "%b %Y"))

customers = pd.read_csv('/home/markaw/heroes/netsuite_cashsales//Customer.csv', encoding='iso-8859-1')
subsidiaries = pd.read_csv('/home/markaw/heroes/netsuite_cashsales//subsidiaries.csv')

while True:
    brand = input('please provide Brand')
    if brand not in subsidiaries['Brand'].tolist():
        print('Please check the spelling for that brand')
        continue
    else:
        break

while True:
    country = input('please provide Country')
    if country not in customers['Country'].tolist():
        print('That country seems incorrect, check spelling')
        continue
    else:
        break

brand_ticker = subsidiaries[subsidiaries['Brand'] == brand].iloc[0][0].upper().replace(" ","")[0:3]
location = 'Amazon FBA - ' + (subsidiaries[subsidiaries['Brand'] ==  brand].iloc[0][1]) +' - '+ (customers[customers['Country'] == country].iloc[0][4])

# create template

final_dict = {
    'ExternalID': f"""{brand_ticker}{date.today().strftime("%d%m%Y")}{customers[customers['Country'] == country].iloc[0][3]}""",
    'tranId':' ',
    'customerRef': customers[customers['Country'] == country].iloc[0][1],
    'tranDate': cs_heroes['tranDate'],
    'postingPeriodRef': cs_heroes['postingPeriodRef'],
    'exchangeRate': '1',
    'locationRef':location,
    'isTaxable': 'TRUE',
    'itemLine_itemRef':cs_heroes['heroes_product_id'],
    'itemLine_quantity':cs_heroes['quantity-purchased'],
    'itemLine_unitsRef':'Units',
    'itemLine_salesPrice':cs_heroes['amount']/cs_heroes['quantity-purchased'],
    'itemLine_priceLevelRef':'Custom',
    'Subsiduary':subsidiaries[subsidiaries['Brand'] == brand].iloc[0][3],
    'Expected Deposit date':(datetime.datetime.strptime(cs_heroes['tranDate'].max(), "%d/%m/%Y") + datetime.timedelta(days=15)).strftime("%d/%m/%Y")
}

final = pd.DataFrame(final_dict)

# export

final = final[final['postingPeriodRef'] =='Jun 2021']


name = f"""/home/markaw/heroes/netsuite_cashsales/cashsale_templates/{brand_ticker}{date.today().strftime("%d%m%Y")}{customers[customers['Country'] == country].iloc[0][3]}.csv"""


if len(final) < 5000:
    final.to_csv(name, index=False)
    
elif 5000 <= len(final) < 10000:
    final[0:5000].to_csv(f'{name}1', index=False)
    
    final2 = final[(final.index >= 5001) & ((final.index < 10000))].copy()
    final2['ExternalID'] = final['ExternalID']+'b'
    final2.to_csv(f'{name}2', index=False)
    
elif len(final) >= 10000:
    final[0:5000].to_csv(name, index=False)
    
    final2 = final[(final.index >= 5001) & ((final.index < 10000))].copy()
    final2['ExternalID'] = final['ExternalID']+'b'
    final2.to_csv(f'{name}2', index=False)
    
    final3 = final[(final.index >= 5001) & ((final.index < 10000))].copy()
    final3['ExternalID'] = final['ExternalID']+'c'
    final3.to_csv(f'{name}3', index=False)

total = (final['itemLine_salesPrice']*final['itemLine_quantity']).sum()
print(f'{len(final)} rows exported! The total sum amount is {total}')

final.head()

please provide the file name you want to import 247774018793
please provide Brand Theraflow
please provide Country United States


15419 rows exported! The total sum amount is 119138.0


,ExternalID,tranId,customerRef,tranDate,postingPeriodRef,exchangeRate,locationRef,isTaxable,itemLine_itemRef,itemLine_quantity,itemLine_unitsRef,itemLine_salesPrice,itemLine_priceLevelRef,Subsiduary,Expected Deposit date
0,THE23062021US,,16 Amazon - US,10/06/2021,Jun 2021,1,Amazon FBA - KGP - US,TRUE,THF-FRD-WW-ZZ-10,1.0,Units,17.95,Custom,2,30/06/2021
1,THE23062021US,,16 Amazon - US,10/06/2021,Jun 2021,1,Amazon FBA - KGP - US,TRUE,THF-FRD-WW-ZZ-10,1.0,Units,1.75,Custom,2,30/06/2021
2,THE23062021US,,16 Amazon - US,09/06/2021,Jun 2021,1,Amazon FBA - KGP - US,TRUE,THF-FRD-WW-ZZ-10,1.0,Units,17.95,Custom,2,30/06/2021
3,THE23062021US,,16 Amazon - US,09/06/2021,Jun 2021,1,Amazon FBA - KGP - US,TRUE,THF-FRD-WW-ZZ-10,1.0,Units,1.49,Custom,2,30/06/2021
4,THE23062021US,,16 Amazon - US,09/06/2021,Jun 2021,1,Amazon FBA - KGP - US,TRUE,THF-FRD-WW-ZZ-10,1.0,Units,-0.90,Custom,2,30/06/2021


In [3]:
cs = df[(df['transaction-type'] == 'Order')&(df['amount-description'] == 'Principal')|(df['amount-description'] == 'Tax')].copy()

,settlement-id,settlement-start-date,settlement-end-date,deposit-date,total-amount,currency,transaction-type,order-id,merchant-order-id,adjustment-id,...,order-item-code,merchant-order-item-id,merchant-adjustment-item-id,quantity-purchased,promotion-id,heroes_product_id,sku,datetime,tranDate,postingPeriodRef


In [7]:
ancilliary = df[(df['transaction-type'] == 'other-transaction')&
                ((df['amount-description'] == 'Principal')|
                (df['amount-description'] == 'REVERSAL_REIMBURSEMENT')|
                (df['amount-description'] == 'WAREHOUSE_DAMAGE')|
                (df['amount-description'] == 'GiftWrap')|
                (df['amount-description'] == 'RestockingFee')|
                (df['amount-description'] == 'GiftWrapTax')|
                (df['amount-description'] == 'Replacement')|
                (df['amount-description'] == 'Warehouse damage')|
                 (df['amount-description'] == 'Discount')|
                 (df['amount-description'] == 'Cupon')|
                 (df['amount-description'] == 'Principal')|
                 (df['amount-description'] == 'Shipping')|
                 (df['amount-description'] == 'Refund')|
                 (df['amount-description'] == 'Discount refund')|
                 (df['amount-description'] == 'FREE_REPLACEMENT_REFUND_ITEMS'))].copy()

ancilliary

,settlement-id,settlement-start-date,settlement-end-date,deposit-date,total-amount,currency,transaction-type,order-id,merchant-order-id,adjustment-id,...,amount,fulfillment-id,posted-date,posted-date-time,order-item-code,merchant-order-item-id,merchant-adjustment-item-id,sku,quantity-purchased,promotion-id
45821,14323110651,NaN,NaN,NaN,NaN,NaN,other-transaction,112-7419476-3075443,NaN,amzn1:crow:qK/5KaHFTZafzJdFTTUb5g,...,3.84,NaN,2021-06-11,2021-06-11 09:33:54 UTC,NaN,NaN,NaN,UC-22HY-A6DX,1.0,NaN
45822,14323110651,NaN,NaN,NaN,NaN,NaN,other-transaction,111-8183836-1610603,NaN,amzn1:crow:yl/8NoWjS9miejp36BcxMg,...,4.63,NaN,2021-06-02,2021-06-02 06:34:07 UTC,NaN,NaN,NaN,Q0-LLT2-BIYO,1.0,NaN
45823,14323110651,NaN,NaN,NaN,NaN,NaN,other-transaction,112-4503396-0618628,NaN,amzn1:crow:F8HCEb7NQV+Xl2QOeA1jow,...,9.54,NaN,2021-06-11,2021-06-11 05:24:47 UTC,NaN,NaN,NaN,Q0-GFBI-B0IJ,1.0,NaN
45824,14323110651,NaN,NaN,NaN,NaN,NaN,other-transaction,NaN,NaN,NaN,...,10.31,NaN,2021-06-09,2021-06-09 09:05:19 UTC,NaN,NaN,NaN,Q0-GFBI-B0IJ,1.0,NaN
45825,14323110651,NaN,NaN,NaN,NaN,NaN,other-transaction,113-4448959-0013848,NaN,amzn1:crow:4aAItWR6QbWnY93Hqbu2bw,...,4.63,NaN,2021-06-04,2021-06-04 03:16:30 UTC,NaN,NaN,NaN,Q0-LLT2-BIYO,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45969,14323110651,NaN,NaN,NaN,NaN,NaN,other-transaction,113-2915213-6040204,NaN,amzn1:crow:kOpacCjqQci3XBc/p5O6uw,...,11.31,NaN,2021-06-05,2021-06-05 19:17:22 UTC,NaN,NaN,NaN,9V-OE5I-MU6W,1.0,NaN
45970,14323110651,NaN,NaN,NaN,NaN,NaN,other-transaction,111-3213370-8191443,NaN,NaN,...,8.17,NaN,2021-06-12,2021-06-12 22:56:27 UTC,NaN,NaN,NaN,8Z-W4S0-2QE3,1.0,NaN
45971,14323110651,NaN,NaN,NaN,NaN,NaN,other-transaction,113-2116189-9782655,NaN,amzn1:crow:Df93ri+WTtqAH1ElA+Zx2w,...,2.50,NaN,2021-06-04,2021-06-04 07:41:52 UTC,NaN,NaN,NaN,UC-22HY-A6DX,1.0,NaN
45972,14323110651,NaN,NaN,NaN,NaN,NaN,other-transaction,113-9444244-7480213,NaN,amzn1:crow:bM71XdpRQAeZWoVVQ59muQ,...,9.54,NaN,2021-06-14,2021-06-14 18:16:27 UTC,NaN,NaN,NaN,Q0-GFBI-B0IJ,1.0,NaN
